In [5]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1Adeu5BWy2EPddJV4MEh03FJ-6ltLQUE4pitJ70A7iV9hNsots7iRiQCKSFA

Successfully saved authorization token.


In [6]:
ee.Initialize()

In [7]:
threeletters = {}
with open('country_codes.csv', 'r') as ifile:
    ilines = ifile.readlines()
    for line in ilines:
        info = line.split(',')
        threeletters[info[0].strip()] = info[1].strip()
        
with open('urbext_orphans_to_LSIB', 'r') as ifile:
    urbet_orphans_to_LSIB = json.loads(ifile.read())

In [8]:
lsib_fc = ee.FeatureCollection("USDOS/LSIB/2017")
lsib_pd = geemap.ee_to_pandas(lsib_fc)
lnames = set(list(lsib_pd['COUNTRY_NA']))

In [5]:
urbext_2020 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents_unions_2020')
urbext_data_2020 = geemap.ee_to_pandas(urbext_2020)
urbext_2015 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents_unions_2015')
urbext_data_2015 = geemap.ee_to_pandas(urbext_2015)

In [9]:
countrycode_to_LSIB = {'AUS': ['Australia'],
 'IRQ': ['Iraq'],
 'SYR': ['Syria'],
 'LBY': ['Libya'],
 'SDN': ['Sudan'],
 'MMR': ['Burma'],
 'THA': ['Thailand'],
 'CI': ["Cote d'Ivoire"],
 'CMR': ['Cameroon'],
 'ETH': ['Ethiopia'],
 'GHA': ['Ghana'],
 'MOZ': ['Mozambique'],
 'TZA': ['Tanzania'],
 'ZMB': ['Zambia'],
 'BGD': ['Bangladesh'],
 'IND': ['India'],
 'ESP': ['Spain'],
 'FRA': ['France'],
 'ITA': ['Italy'],
 'KP': ['Korea, North'],
 'KR': ['Korea, South'],
 'CHN': ['China'],
 'JPN': ['Japan'],
 'SAU': ['Saudi Arabia'],
 'TUR': ['Turkey'],
 'DZA': ['Algeria'],
 'EGY': ['Egypt'],
 'MAR': ['Morocco'],
 'IDN': ['Indonesia'],
 'MYS': ['Malaysia'],
 'PHL': ['Philippines'],
 'VNM': ['Vietnam'],
 'CD': ['Congo, Dem Rep of the'],
 'NGA': ['Nigeria'],
 'ZAF': ['South Africa'],
 'BRA': ['Brazil'],
 'ALB': ['Albania'],
 'BGR': ['Bulgaria'],
 'GRC': ['Greece'],
 'HRV': ['Croatia'],
 'MKD': ['Macedonia'],
 'MLT': ['Malta'],
 'MNE': ['Montenegro'],
 'PRT': ['Portugal'],
 'SRB': ['Serbia'],
 'CYP': ['Cyprus'],
 'ISR': ['Israel'],
 'JOR': ['Jordan'],
 'LBN': ['Lebanon'],
 'PSE': ['Gaza Strip (disp)', 'West Bank (disp)'],
 'YEM': ['Yemen'],
 'ESH': ['Western Sahara (disp)'],
 'TUN': ['Tunisia'],
 'TL': ['Timor-Leste'],
 'KHM': ['Cambodia'],
 'LAO': ['Laos'],
 'SGP': ['Singapore'],
 'CG': ['Congo, Rep of the'],
 'AGO': ['Angola'],
 'BDI': ['Burundi'],
 'BEN': ['Benin'],
 'BFA': ['Burkina Faso'],
 'BWA': ['Botswana'],
 'CAF': ['Central African Rep'],
 'CPV': ['Cabo Verde'],
 'DJI': ['Djibouti'],
 'ERI': ['Eritrea'],
 'GAB': ['Gabon'],
 'GIN': ['Guinea'],
 'GMB': ['Gambia, The'],
 'GNB': ['Guinea-Bissau'],
 'GNQ': ['Equatorial Guinea'],
 'KEN': ['Kenya'],
 'LBR': ['Liberia'],
 'LSO': ['Lesotho'],
 'MDG': ['Madagascar'],
 'MLI': ['Mali'],
 'MRT': ['Mauritania'],
 'MUS': ['Mauritius'],
 'MWI': ['Malawi'],
 'NAM': ['Namibia'],
 'NER': ['Niger'],
 'REU': ['Reunion (Fr)'],
 'RWA': ['Rwanda'],
 'SEN': ['Senegal'],
 'SLE': ['Sierra Leone'],
 'SOM': ['Somalia'],
 'SSD': ['South Sudan'],
 'TCD': ['Chad'],
 'TGO': ['Togo'],
 'UGA': ['Uganda'],
 'ZWE': ['Zimbabwe'],
 'IRL': ['Ireland'],
 'SVK': ['Slovakia'],
 'SVN': ['Slovenia'],
 'BIH': ['Bosnia & Herzegovina'],
 'ARM': ['Armenia'],
 'QAT': ['Qatar'],
 'OMN': ['Oman'],
 'TJK': ['Tajikistan'],
 'KGZ': ['Kyrgyzstan'],
 'MDA': ['Moldova'],
 'AZE': ['Azerbaijan'],
 'DNK': ['Denmark'],
 'GEO': ['Georgia'],
 'AUT': ['Austria'],
 'LTU': ['Lithuania'],
 'TKM': ['Turkmenistan'],
 'ARE': ['United Arab Emirates'],
 'LKA': ['Sri Lanka'],
 'NOR': ['Norway'],
 'CZE': ['Czechia'],
 'FIN': ['Finland'],
 'AFG': ['Afghanistan'],
 'HUN': ['Hungary'],
 'NPL': ['Nepal'],
 'CHE': ['Switzerland'],
 'SWE': ['Sweden'],
 'BEL': ['Belgium'],
 'BLR': ['Belarus'],
 'UZB': ['Uzbekistan'],
 'ROU': ['Romania'],
 'KAZ': ['Kazakhstan'],
 'NLD': ['Netherlands'],
 'POL': ['Poland'],
 'GBR': ['United Kingdom'],
 'DEU': ['Germany'],
 'IRN': ['Iran'],
 'ISL': ['Iceland'],
 'LVA': ['Latvia'],
 'EST': ['Estonia'],
 'KWT': ['Kuwait'],
 'BHR': ['Bahrain'],
 'PAK': ['Pakistan'],
 'RUS': ['Russia'],
 'USA': ['United States'],
 'CAN': ['Canada'],
 'UKR': ['Ukraine'],
 'CUB': ['Cuba'],
 'ECU': ['Ecuador'],
 'PER': ['Peru'],
 'CHL': ['Chile'],
 'ARG': ['Argentina'],
 'COL': ['Colombia'],
 'MEX': ['Mexico'],
 'VEN': ['Venezuela'],
 'GUM': ['Guam (US)'],
 'MNG': ['Mongolia'],
 'NCL': ['New Caledonia (Fr)'],
 'PNG': ['Papua New Guinea'],
 'FJI': ['Fiji'],
 'NZL': ['New Zealand'],
 'JAM': ['Jamaica'],
 'GLP': ['Guadeloupe (Fr)'],
 'GUF': ['French Guiana (Fr)'],
 'HTI': ['Haiti'],
 'DOM': ['Dominican Republic'],
 'PAN': ['Panama'],
 'CUW': ['Curacao (Neth)'],
 'HND': ['Honduras'],
 'PRY': ['Paraguay'],
 'SUR': ['Suriname'],
 'GTM': ['Guatemala'],
 'NIC': ['Nicaragua'],
 'CRI': ['Costa Rica'],
 'SLV': ['El Salvador'],
 'PRI': ['Puerto Rico (US)'],
 'BHS': ['Bahamas, The'],
 'URY': ['Uruguay'],
 'BOL': ['Bolivia'],
 'GUY': ['Guyana'],
 'AND': ['Andorra'],
 'BRN': ['Brunei'],
 'BTN': ['Bhutan'],
 'BLZ': ['Belize'],
 'GIB': ['Gibraltar (UK)'],
 'HKG': ['Hong Kong (Ch)'],
 'LIE': ['Liechtenstein'],
 'LUX': ['Luxembourg'],
 'MCO': ['Monaco'],
 'MAF': ['St Martin (Fr)'],
 'MAC': ['Macau (Ch)'],
 'SMR': ['San Marino'],
 'SXM': ['Sint Maarten (Neth)'],
 'SWZ': ['Swaziland'],
 'VAT': ['Vatican City'],   
    'PT-30':['Portugal [Madeira Is]'],
                      'PT-20': ['Portugal [Azores]']
                      }

In [10]:
LSIB_to_code3 = {}
for i, j in countrycode_to_LSIB.items():
    for k in j:
        if i in threeletters:
            i3 = threeletters[i]
        else:
            i3 = i
        LSIB_to_code3[k] = i3

In [11]:
import json
with open('LSIB_to_code.json', 'w') as ofile:
    ofile.write(json.dumps(LSIB_to_code3))

In [12]:
with open('more_countries.txt', 'r') as ifile:
    for line in ifile.readlines():
        items = [i.strip().replace('"','') for i in line.split('\t')]
        LSIB_to_code3[items[0]] = items[1]

In [13]:
for n in lnames:
    if not n in LSIB_to_code3:
        LSIB_to_code3[n] = n

In [14]:
lsib_to_code = ee.Dictionary(LSIB_to_code3)

In [15]:
def makeFeatures(ufc, upd, year, idx):
    res_fc = ee.FeatureCollection([])
    print(year)
    for i in list(range(len(upd)))[(idx*500): (idx+1)*500]:
        ua = upd.iloc[i]
        ua_f = ufc.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        ua_geom = ua_f.geometry()
        countries_fc = lsib_fc.filter(ee.Filter.intersects('.geo', ua_geom))
        def makeFeature(c_f):
            geom = c_f.geometry().intersection(ua_geom)
            properties = ee.Dictionary({
                'city_ids': str(ua['city_ids']),
                'country': lsib_to_code.get(c_f.get('COUNTRY_NA')),
                'year': year
            })
            return ee.Feature(geom, properties)
        res_fc = res_fc.merge(countries_fc.map(makeFeature))
        if i % 100 == 0:
            print(i, end=' ')
    return res_fc





In [20]:
for year in [1990, 2000, 2005, 2010, 2015, 2020]:
    for idx in range(8):
        urbext = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents_unions_v6_{}'.format(year))
        urbext_data = geemap.ee_to_pandas(urbext)
        fc = makeFeatures(urbext, urbext_data, year, idx)
        task = ee.batch.Export.table.toAsset(
            collection = fc,
            description = 'countrycities_{0}_{1}'.format(year, idx),
            assetId = 'projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}_{1}'.format(year, idx)
        )
        task.start()

1990
0 100 200 300 400 1990
500 600 700 800 900 1990
1000 1100 1200 1300 1400 1990
1500 1600 1700 1800 1900 1990
2000 2100 2200 2300 2400 1990
2500 2600 2700 2800 2900 1990
3000 3100 3200 3300 3400 1990
3500 3600 3700 3800 3900 2000
0 100 200 300 400 2000
500 600 700 800 900 2000
1000 1100 1200 1300 1400 2000
1500 1600 1700 1800 1900 2000
2000 2100 2200 2300 2400 2000
2500 2600 2700 2800 2900 2000
3000 3100 3200 3300 3400 2000
3500 3600 3700 3800 3900 2005
0 100 200 300 400 2005
500 600 700 800 900 2005
1000 1100 1200 1300 1400 2005
1500 1600 1700 1800 1900 2005
2000 2100 2200 2300 2400 2005
2500 2600 2700 2800 2900 2005
3000 3100 3200 3300 3400 2005
3500 3600 3700 3800 3900 2010
0 100 200 300 400 2010
500 600 700 800 900 2010
1000 1100 1200 1300 1400 2010
1500 1600 1700 1800 1900 2010
2000 2100 2200 2300 2400 2010
2500 2600 2700 2800 2900 2010
3000 3100 3200 3300 3400 2010
3500 3600 3700 3800 3900 2015
0 100 200 300 400 2015
500 600 700 800 900 2015
1000 1100 1200 1300 1400 2015
1500 

In [21]:
for YEAR in [1990, 2000, 2005, 2010, 2015, 2020]:
    fc = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}_0'.format(YEAR))
    for idx in range(1, 8):
        fc_next = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}_{1}'.format(YEAR, idx))
        fc = fc.merge(fc_next)
    task = ee.batch.Export.table.toAsset(
        collection = fc,
        description = 'countrycities_{0}'.format(YEAR),
        assetId = 'projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}'.format(YEAR)
    )
    task.start()